In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from img2vec_dino2 import Img2VecResnet18
from tqdm import tqdm
from PIL import Image

from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.patches import Ellipse
import glob

from transformers import AutoImageProcessor, AutoModel
import torch


In [ ]:
CROPS_PATH = "../data/knowledge_base/crops/object/"
IMG_PATH = "../data/img/cocacola_bottle.jpeg"

# Functions

In [ ]:
# Function to display images on the scatter plot
def show_images(x, y, imagenes, ax):
    for i in range(len(imagenes)):
        # Create an image box for each image using OffsetImage
        image_box = OffsetImage(imagenes[i], zoom=0.6)

        # Create an annotation box for each image at the corresponding coordinates
        ab = AnnotationBbox(image_box, (x[i], y[i]), frameon=False)

        # Add the annotation box to the plot
        ax.add_artist(ab)

# Get embeddings

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "facebook/dinov2-base"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

def get_dino_embedding(image_path: str):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        emb = outputs.last_hidden_state[:, 0, :]  # CLS token
        emb = torch.nn.functional.normalize(emb, dim=-1)
    return emb.squeeze().cpu().numpy()

image_paths = glob.glob(f"{CROPS_PATH}/**/*.jpg")

# Ejemplo: obtener embeddings de todas las imágenes
embeddings = []
for path in image_paths:
    emb = get_dino_embedding(path)
    embeddings.append(emb)

# Add a new image not belonging to dataset
emb = get_dino_embedding(IMG_PATH)
embeddings.append(emb)

embeddings = np.vstack(embeddings)

# TSNE

In [ ]:
# Perform t-SNE dimensionality reduction on the vector representations
tsne = TSNE(n_components=2, random_state=42)
embeddings_tsne = tsne.fit_transform(embeddings)

# Get thumbnails

In [ ]:
# Create an empty list to store the thumbnail images
images = []

# Iterate over all files in the directory specified by PATH
for image in tqdm(image_paths):
    # Open each image file
    I = Image.open(image)

    # Resize the image to a thumbnail size of [100, 100] using Lanczos resampling
    I.thumbnail([100, 100], Image.Resampling.LANCZOS)

    # Append the resized image to the images list
    images.append(I)


# Open the image 'cocacola.jpeg'
I = Image.open(IMG_PATH)
# Resize the image to a thumbnail size of [100, 100] using Lanczos resampling
I.thumbnail([100, 100], Image.Resampling.LANCZOS)
# Append the resized image to the imagenes list
images.append(I)

# Plot T-SNE

In [ ]:
# Create a scatter plot to visualize the t-SNE embeddings
fig, ax = plt.subplots(figsize=(20, 16))
scatter = ax.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1])

# Call the function to display the images on the scatter plot
show_images(embeddings_tsne[:, 0], embeddings_tsne[:, 1], images, ax)


# Highlight the new added picture
# Create an Ellipse patch with specified center, width, height, and angle
ellipse = Ellipse(xy=embeddings_tsne[-1,:], width=0.6, height=1.6, angle=0, fill=False)
# Add the Ellipse patch to the axes
ax.add_patch(ellipse)

# Set the title and labels for the plot
ax.set_title('t-SNE')
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')

# Display the plot
plt.show()